In [6]:
%load_ext autoreload
%autoreload 2
# import modules.dynpy as dynpy
# import importlib
# importlib.reload(dynpy)
# from modules.dynpy import *


from dynamics import *


# import ode_tools
# importlib.reload(ode_tools)
# from ode_tools import *

# import modules.timeseries as timeseries
# importlib.reload(timeseries)
# from modules.timeseries import *

from sympy import *
from sympy.physics.mechanics import *
import sympy as sym
import numpy as np
import numpy.fft as fft

import matplotlib.pyplot as plt
#import pandas as pd

mechanics_printing(pretty_print=True)

m_v,m_p,A_y,A_z,Phi_y,Phi_z,omega,k_c,c,l_w,l_c,k_w = symbols('m_v m_mod_p A_y A_z Phi_y Phi_z omega k_mod_c c l_w l_c k_mod_w')
m_c,m_p,u_0,u_c0,l_0,l_c,omega,g = symbols('m_mod_c,m_mod_p,u_mod_0,u_mod_c0,l_0,l_c,omega g',positive=True)

omega_c,omega_p,omega_m,omega_d,omega_sw=symbols('omega_c,omega_p,omega_m,omega_d, omega_sw',positive=True)

A1,A2,eps=symbols('A_1,A_2 ,epsilon',real=True)

invars_dict = {g:9.81,
               m_c:1e4,
               k_w:4.09e6
              }

params_dict = {A_y:0.5,
               A_z:1.5,
               Phi_y:-2.396,#0,
               Phi_z:0.315,#np.pi/2,
               omega:float(((1/7)*2*pi).n()),
               k_c:2.5e6,
               c:0,
               l_w:float((((1/7*2*pi *2 ).n() **2*(1-0.5*1.5/48)  )**-1*9.81*4)),
               l_c:0,
               m_p:1e5,
              }



#3 dof
t = symbols('t')
coords='phi h h_c'
phi, h, h_c = dynamicsymbols('varphi h_mod h_mod_c')
dphi, dh, dh_c = dynamicsymbols('varphi h_mod h_mod_c', 1)
q = Matrix([phi, h, h_c])
u=q.diff(t)
du = q[3:6]
print('du')
nDOF=len(u)
print(q)
print('u0',u[0])
params_list=[m_p,omega]


######################################################### testing

e = Symbol('varepsilon')

m1, m2, k, omega, omega0, A1, A2,delta = symbols('m_1 m_2 k omega omega0 A1 A2 delta',
                                           positive=True)
x1, x2 = dynamicsymbols('x_1 x_2')

Ek = S.One * m1 * x1.diff(t)**2 + S.One / 2 * m1 * x2.diff(t)**2
Ep = S.One * k * x1**2 + 0 * S.One * k * (
    x1 - x2)**2 + S.One * (k+e*(delta+cos(2*sqrt(2*k/m1) * t))) * x2**2 + e * x1 * cos(omega * t) + (
        e / 4 * x1**4 + e * 0 * x1 * x2**3)

test_sys = HarmonicOscillator((Ek - Ep).expand(), [x1, x2],
                              label='Test system')
display(test_sys._eoms)

# sys_sol=LinearODESolution(test_sys.subs(k,omega0**2*m1,method='direct'),t,[x1,x2])
# sys_sol.solution()

#sys_sol=WeakNonlinearProblemSolution(test_sys.subs(k,(omega**2+A1*e+A2*e**2)*m1),t,[x1,x2],eps=Symbol('e'))
# sys_sol = WeakNonlinearProblemSolution(test_sys._eoms,
#                                        t, [x1, x2],
#                                        eps=e,
#                                        omega=omega * sqrt(m1 / k),
#                                        params_values={
#                                            e: 1.5,
#                                            m1: 10,
#                                            k: 20,
#                                            omega: 1.0,
#                                            Symbol('C1'): 0,
#                                            Symbol('C2'): 0.1,
#                                            Symbol('C3'): 0.1,
#                                            Symbol('C4'): 0.1,
#                                             Symbol('A_00_1'): 0.2,
#                                             Symbol('A_00_2'): 0.2,
#                                             Symbol('A_11_1'): 0.2,
#                                             Symbol('A_11_2'): 0.2,
#                                        }
#                                       )



sys_sol = MultiTimeScaleMethod(test_sys._eoms,
                                       t, [x1, x2],
                                       eps=e,
                                       omega=omega,
                                       params_values={
                                           e: 1.5,
                                           m1: 10,
                                           k: 20,
                                           omega: 1.0,
                                           Symbol('C1'): 0,
                                           Symbol('C2'): 0.1,
                                           Symbol('C3'): 0.1,
                                           Symbol('C4'): 0.1,
                                            Symbol('A_00_1'): 0.2,
                                            Symbol('A_00_2'): 0.2,
                                            Symbol('A_11_1'): 0.2,
                                            Symbol('A_11_2'): 0.2,
                                       }
                                      )

#display(sys_sol.predicted_solution(order=5,dict=True))
display(sys_sol.approximation_function(1))

# display(sys_sol.zeroth_approximation(equation=True))
# display(sys_sol.zeroth_approximation(dict=True))
# display(sys_sol.first_approximation())

#display(sys_sol.nth_approximation(2))

display(*sys_sol.eoms_approximation(1).doit().expand())
#sys_sol.nth_eoms_approximation(0)
print('+++')
#sys_sol.eoms_approximation(3).applyfunc(lambda obj: obj.diff(e,0)).subs(e,0)
#sys_sol.zeroth_approximation(dict=True)
#sys_sol.nth_approximation(1,dict=True)
display(sys_sol.nth_order_solution(1))
#sys_sol.eigenvalue_approximation(order=4)
#sys_sol.nth_eoms_approximation(0),sys_sol.nth_eoms_approximation(1),sys_sol.nth_eoms_approximation(2),sys_sol.nth_eoms_approximation(3)
#display(sys_sol(t, order=1)[0])
# sys_sol(np.linspace(0,10,200),order=3).plot()

# sys_sol(np.linspace(0,10,200),order=1).plot()

######################################### testing



  

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


ImportError: attempted relative import with no known parent package